In [43]:
import csv
import numpy as np 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn import preprocessing

def get_list(file_name):
    big_list = []
    id_list = []
    missing_list = []
    clean = []
    mask = [1,3,1,3,3,1,1,3,2,3,2,2,4,2]
    with open(file_name) as csv_file:
        csv_reader = csv.reader(csv_file,delimiter=',')
        cnt = False
        crap = []
        mxn = 0
        row_cnt = 0
        rx = {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7,'H':8,'I':9,'J':10,'K':11,'L':12,'M':13,'N':14,'O':15}
        for row in csv_reader:
            if cnt :
                temp_row = []
                missing = False 
                for counter in range(len(row)) :
                    if row[counter]!='':
                        if mask[counter]==1:
                            if counter==0:
                                id_list.append(str(row[counter]))
                            elif counter==2 or counter ==5:
                                temp_row.append(rx[row[counter]])
                            else :
                                num = rx[row[counter]]
                                for i in range(1,15):
                                    if i == num :
                                        temp_row.append(1)
                                    else :
                                        temp_row.append(0)
                        elif mask[counter]==2:
                            temp_row.append(int(row[counter]))
                        elif mask[counter]==3:             
                            temp_row.append(float(row[counter]))
                        else :                           
                            if str(row[counter])=='Male':
                                temp_row.append(1)
                                temp_row.append(0)
                            else:
                                temp_row.append(0)
                                temp_row.append(1)
                    else :
                        missing =True
                        temp_row.append(-1)
                if missing :
                    missing_list.append(row_cnt)
                else :
                    clean.append(temp_row)
                big_list.append(temp_row)
                row_cnt = row_cnt+1
            cnt = True
    clean = np.array(clean,dtype='float64')
    big_list= np.array(big_list,dtype='float64')
    missing_list = np.array(missing_list,dtype='int64')
    id_list = np.array(id_list)
    return (clean, id_list, missing_list, big_list)

def get_data(input_data, attr_list):
    return input_data[:,attr_list]

def get_accuracy(list1,list2,foo):
    if foo:
        acc = 0 
        for i in range(list1.shape[0]):
            if list1[i]==list2[i] :
                acc = acc +1
        return (float(acc)/list1.shape[0])
    else :
        acc = 0
        for i in range(list1.shape[0]):
            acc = acc+(list1[i]-list2[i])**2
        acc = acc/list1.shape[0]
        return acc 

def give_accuracy(X,Y,clf,b):
    trainX = X[:int(0.8*X.shape[0]),:]
    testX = X[int(0.8*X.shape[0]):,:]
    trainY = Y[:int(0.8*Y.shape[0])]
    testY = Y[int(0.8*Y.shape[0]):]
    clf.fit(trainX,trainY)
    print("Accuracy :",get_accuracy(testY,clf.predict(testX),b))
    
attr_miss = [1,2,3,4,21]
attr_X = [0,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25]

# didn't keep 26
clean_train,train_id,train_miss, s1_train = get_list('train.csv')
clean_test,test_id,test_miss, s1_test = get_list('test.csv')

train = np.concatenate((clean_train[:,:-1],clean_test),axis=0)

my_train = s1_train 
my_test = s1_test 

# global variables
cleanX = get_data(train,attr_X)
test1_X = get_data(s1_train,attr_X)
test2_X = get_data(s1_test,attr_X)

attr1_Y = [1]
cleanY = get_data(train,attr1_Y)
clf1 = GradientBoostingClassifier(n_estimators=300)
print("Fitting RandomForestClassifier")
clf1.fit(cleanX,cleanY)
column1 = clf1.predict(test1_X)
column2 = clf1.predict(test2_X)
for cnt in train_miss:
    if(my_train[cnt][1]==-1):
        my_train[cnt][1]=column1[cnt]
print("Accuracy 1 :",get_accuracy(cleanY,clf1.predict(cleanX),True))
for cnt in test_miss :
    if(my_test[cnt][1]==-1):
        my_test[cnt][1] = column2[cnt]
print("Done column1")

# for 2 linear regression problem  ?
attr1_Y = [2]
cleanY = get_data(train,attr1_Y)
clf2 = Ridge()
print("Fitting Linear Regression")
clf2.fit(cleanX,cleanY)
column1 = clf2.predict(test1_X)
column2 = clf2.predict(test2_X)
for cnt in train_miss:
    if(my_train[cnt][2]==-1):
        my_train[cnt][2]=column1[cnt]
print("Error 2 :",get_accuracy(cleanY,clf2.predict(cleanX),False))
for cnt in test_miss :
    if(my_test[cnt][2]==-1):
        my_test[cnt][2] = column2[cnt]
print("Done column2")

attr1_Y = [3]
cleanY = get_data(train,attr1_Y)
clf3 = Ridge()
print("Fitting Linear Regression")
clf3.fit(cleanX,cleanY)
column1 = clf3.predict(test1_X)
column2 = clf3.predict(test2_X)
for cnt in train_miss:
    if(my_train[cnt][3]==-1):
        my_train[cnt][3]=column1[cnt]
print("Error 3 :",get_accuracy(cleanY,clf3.predict(cleanX),False))
for cnt in test_miss :
    if(my_train[cnt][3]==-1):
        my_test[cnt][3] = column2[cnt]
print("Done column3")

attr1_Y =[4]
cleanY = get_data(train,attr1_Y)
clf4 = GradientBoostingClassifier(n_estimators=300)
print("Fitting RandomForestClassifier")
clf4.fit(cleanX,cleanY)
column1 = clf4.predict(test1_X)
column2 = clf4.predict(test2_X)
for cnt in train_miss:
    if(my_train[cnt][4]==-1):
        my_train[cnt][4]=column1[cnt]
print("Accuracy 4 :",get_accuracy(cleanY,clf4.predict(cleanX),True))
for cnt in test_miss :
    if(my_train[cnt][4]==-1):
        my_test[cnt][4] = column2[cnt]
print("Done column4")

attr1_Y =[21]
cleanY = get_data(train,attr1_Y)
clf5 = Ridge()
print("Fitting Ridge Regression")
clf5.fit(cleanX,cleanY)
column1 = clf5.predict(test1_X)
column2 = clf5.predict(test2_X)
for cnt in train_miss:
    if(my_train[cnt][21]==-1):
        my_train[cnt][21]=column1[cnt]
print("Error 5 :",get_accuracy(cleanY,clf5.predict(cleanX),False))
for cnt in test_miss :
    if(my_test[cnt][21]==-1):
        my_test[cnt][21] = column2[cnt]
print("Done column21")

def give_final(final_data):
    data = []
    for x in final_data:
        row_cnt = 0
        temp = []
        for row_cnt,elem in enumerate(x):
            if(row_cnt==1):
                if(elem==1):
                    temp.extend([1,0,0,0,0])
                elif(elem==2):
                    temp.extend([0,1,0,0,0])
                elif(elem==3) :
                    temp.extend([0,0,1,0,0])
                elif(elem==4):
                    temp.extend([0,0,0,1,0])
                else :
                    temp.extend([0,0,0,0,1])
            elif(row_cnt==4):
                if(elem==1):
                    temp.extend([1,0,0])
                elif(elem==2):
                    temp.extend([0,1,0])
                else :
                    temp.extend([0,0,1])
            else :
                temp.append(elem)
        data.append(temp)
    return np.array(data)

final_trainX = give_final(my_train[:,:-1])
final_trainY = my_train[:,-1:]
final_testX = give_final(my_test)

scale_trainX = preprocessing.scale(final_trainX)
scale_testX = preprocessing.scale(final_testX)

final_clf = GradientBoostingClassifier(n_estimators=1500)
final_clf.fit(scale_trainX,final_trainY)
prediction = final_clf.predict(scale_testX)
answer = []
answer.append(['id','pricing_category'])
for cnt,x in enumerate(prediction) :
    temp = []
    temp.append(test_id[cnt])
    temp.append(x)
    answer.append(temp)
with open('impute.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(answer)

Fitting RandomForestClassifier


/Users/re1nth/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy 1 : 0.37540857527398575
Done column1
Fitting Linear Regression
Error 2 : [12.86274604]
Done column2
Fitting Linear Regression
Error 3 : [0.02990917]
Done column3
Fitting RandomForestClassifier


/Users/re1nth/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy 4 : 0.5210776773697365
Done column4
Fitting Ridge Regression
Error 5 : [471.83449]
Done column21


/Users/re1nth/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
